<a href="https://colab.research.google.com/github/Mingze-L/Final-project/blob/main/NN_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect to the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# set the directory
%cd /content/gdrive/MyDrive/FinalProject/

/content/gdrive/MyDrive/FinalProject


In [ ]:
import pandas as pd

fm_params = pd.read_csv('fm_params.csv')
fm_params_1 = pd.read_csv('fm_params_1.csv')
fm_params_2 = pd.read_csv('fm_params_2.csv')
fm_params_3 = pd.read_csv('fm_params_3.csv')
fm_params_4 = pd.read_csv('fm_params_4.csv')
fm_params_5 = pd.read_csv('fm_params_5.csv')

m_params = pd.read_csv('m_params.csv')
m_params_1 = pd.read_csv('m_params_1.csv')
m_params_2 = pd.read_csv('m_params_2.csv')
m_params_3 = pd.read_csv('m_params_3.csv')
m_params_4 = pd.read_csv('m_params_4.csv')
m_params_5 = pd.read_csv('m_params_5.csv')

t_params = pd.read_csv('t_params.csv')
t_params_1 = pd.read_csv('t_params_1.csv')
t_params_2 = pd.read_csv('t_params_2.csv')
t_params_3 = pd.read_csv('t_params_3.csv')
t_params_4 = pd.read_csv('t_params_4.csv')
t_params_5 = pd.read_csv('t_params_5.csv')

## Time lag

In [ ]:
# lag 5(change to 1 is using lag 1 )
lags = 5
for i in range(1, lags + 1):
    fm_params['lag_' + str(i)] = fm_params['K2'].shift(i)
    fm_params_1['lag_' + str(i)] = fm_params_1['K2'].shift(i)
    fm_params_2['lag_' + str(i)] = fm_params_2['K2'].shift(i)
    fm_params_3['lag_' + str(i)] = fm_params_3['K2'].shift(i)
    fm_params_4['lag_' + str(i)] = fm_params_4['K2'].shift(i)
    fm_params_5['lag_' + str(i)] = fm_params_5['K2'].shift(i)

    m_params['lag_' + str(i)] = m_params['K2'].shift(i)
    m_params_1['lag_' + str(i)] = m_params_1['K2'].shift(i)
    m_params_2['lag_' + str(i)] = m_params_2['K2'].shift(i)
    m_params_3['lag_' + str(i)] = m_params_3['K2'].shift(i)
    m_params_4['lag_' + str(i)] = m_params_4['K2'].shift(i)
    m_params_5['lag_' + str(i)] = m_params_5['K2'].shift(i)

    t_params['lag_' + str(i)] = t_params['K2'].shift(i)
    t_params_1['lag_' + str(i)] = t_params_1['K2'].shift(i)
    t_params_2['lag_' + str(i)] = t_params_2['K2'].shift(i)
    t_params_3['lag_' + str(i)] = t_params_3['K2'].shift(i)
    t_params_4['lag_' + str(i)] = t_params_4['K2'].shift(i)
    t_params_5['lag_' + str(i)] = t_params_5['K2'].shift(i)

In [ ]:
# mark for the autoformer
df_stamp = fm_params[['Year']][:]
df_stamp['Year'] = pd.to_datetime(df_stamp.Year,format = '%Y')

def extract_year(date_obj):
    return date_obj.year

df_stamp['Year'] = df_stamp['Year'].apply(extract_year)
df_stamp['Year'] = (df_stamp['Year'].astype(int)-1922) / (2020-1922) - 0.5
for i in range(1, lags + 1):
  df_stamp['lag_' + str(i)] = df_stamp['Year'].shift(i)

In [ ]:
# Train set
split_index_fm = int(len(fm_params) * 0.8)
split_index_m = int(len(m_params) * 0.8)
split_index_t = int(len(t_params) * 0.8)

fm = fm_params[:split_index_fm]
m = m_params[:split_index_m]
t = t_params[:split_index_t]

split_index_fm_v = int(len(fm) * 0.8)
split_index_m_v = int(len(m) * 0.8)
split_index_t_v = int(len(t) * 0.8)

train_fm = fm[:split_index_fm_v]
train_fm_1 = fm_params_1[:split_index_fm_v]
train_fm_2 = fm_params_2[:split_index_fm_v]
train_fm_3 = fm_params_3[:split_index_fm_v]
train_fm_4 = fm_params_4[:split_index_fm_v]
train_fm_5 = fm_params_5[:split_index_fm_v]

train_m = m[:split_index_m_v]
train_m_1 = m_params_1[:split_index_m_v]
train_m_2 = m_params_2[:split_index_m_v]
train_m_3 = m_params_3[:split_index_m_v]
train_m_4 = m_params_4[:split_index_m_v]
train_m_5 = m_params_5[:split_index_m_v]

train_t = t[:split_index_t_v]
train_t_1 = t_params_1[:split_index_t_v]
train_t_2 = t_params_2[:split_index_t_v]
train_t_3 = t_params_3[:split_index_t_v]
train_t_4 = t_params_4[:split_index_t_v]
train_t_5 = t_params_5[:split_index_t_v]

# Validation set
valid_fm = fm[split_index_fm_v:]
valid_m = m[split_index_m_v:]
valid_t = t[split_index_t_v:]

# Test set
test_fm = fm_params[split_index_fm:]
test_m = m_params[split_index_m:]
test_t = t_params[split_index_t:]

In [ ]:
# split the mark
train_mark = df_stamp[:split_index_fm_v]
valid_mark = df_stamp[split_index_fm_v : split_index_fm]
test_mark = df_stamp[split_index_fm:]

## Model

In [ ]:
# LSTM
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, hidden_size, activation, num_layers, drop_prob):
        super(LSTM,self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(1, hidden_size,num_layers, batch_first = True, dropout=drop_prob if num_layers > 1 else 0.)
        self.fc = nn.Linear(hidden_size,1)
        self.act = activation()

    def forward(self, x):
        x,_ = self.rnn(x)
        x = self.fc(x[:,-1:,:])
        x = self.act(x)
        x = x.squeeze(2)

        return x


In [ ]:
# LTSF-linear
class LTSF(nn.Module):
    def __init__(self):
      super(LTSF, self).__init__()
      self.fc = nn.Linear(1,1)

    def forward(self, x):
      #X(batch_size, 5, 1)

      x = self.fc(x.permute(0,2,1)).permute(0,2,1) #(batch_size, 1, 1)
      x = x.squeeze(2) #(batch_size, 1)

      return x

In [ ]:
# LTSF-Dlinear

class moving_avg(nn.Module):
    def __init__(self):
      super(moving_avg, self).__init__()
      self.kernel_size = 3
      self.avg = nn.AvgPool1d(kernel_size=3, stride=1, padding = 0)

    def forward(self, x):
      # padding on the both ends of time series
      front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
      end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
      x = torch.cat([front, x, end], dim=1)
      x = self.avg(x.permute(0, 2, 1))
      x = x.permute(0, 2, 1)

      return x

class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg()

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class LTSF_D(nn.Module):
    def __init__(self):
      super(LTSF_D, self).__init__()
      self.decompsition = series_decomp()

      self.Linear_Seasonal = nn.Linear(5,1)
      self.Linear_Trend = nn.Linear(5,1)

    def forward(self, x):
      #X(batch_size, 5, 1)

      seasonal_init, trend_init = self.decompsition(x)
      seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1) #(batch_size, 1, 5)

      seasonal_output = self.Linear_Seasonal(seasonal_init) #(batch_size, 1, 1)
      trend_output = self.Linear_Trend(trend_init)

      x = seasonal_output + trend_output #(batch_size, 1, 1)
      x = x.squeeze(2) #(batch_size, 1)

      return x

In [ ]:
# LTSF-Nlinear

class LTSF_N(nn.Module):
    def __init__(self, activation):
      super(LTSF_N, self).__init__()
      self.fc = nn.Linear(3,1)
      self.act = activation()

    def forward(self, x):
      #X(batch_size, 5, 1)
      seq_last = x[:,-1:,:].detach()
      x = x - seq_last

      x = self.fc(x.permute(0,2,1)).permute(0,2,1) #(batch_size, 1, 1)
      x = x + seq_last
      x = self.act(x)
      x = x.squeeze(2) #(batch_size, 1)

      return x

In [ ]:
# bi-direction LSTM

class biLSTM(nn.Module):
    def __init__(self, hidden_size, num_layers, drop_prob, activation):
        super(biLSTM,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(1, hidden_size, num_layers, batch_first = True,
                           bidirectional = True,
                           dropout=drop_prob if num_layers > 1 else 0.)
        self.fc = nn.Linear(2*hidden_size,1)
        self.act = activation()

    def forward(self, x):
      #X(batch_size, 5, 1)
        x,_ = self.rnn(x) #(batch_size, 5, 2*hidden_size)

        x = x[:, -1:, :] #(batch_size, 1, 2*hidden_size)
        x = self.fc(x) #(batch_size,1,1)
        x = self.act(x)
        x = x.squeeze(2) #(batch_size, 1)

        return x

In [ ]:
# Autoformer
from layers.Embed import DataEmbedding, DataEmbedding_wo_pos
from layers.AutoCorrelation import AutoCorrelation, AutoCorrelationLayer
from layers.Autoformer_EncDec import Encoder, Decoder, EncoderLayer, DecoderLayer, my_Layernorm, series_decomp
import math
import numpy as np


class Autoformer(nn.Module):
    """
    Autoformer is the first method to achieve the series-wise connection,
    with inherent O(LlogL) complexity
    """
    def __init__(self, d_model, n_heads, d_ff):
        super(Autoformer, self).__init__()

        # Decomp
        self.decomp = series_decomp(3)

        # Embedding
        # The series-wise connection inherently contains the sequential information.
        # Thus, we can discard the position embedding of transformers.
        self.enc_embedding = DataEmbedding_wo_pos(1, d_model, 'timeF', 'm', 0.1)
        self.dec_embedding = DataEmbedding_wo_pos(1, d_model, 'timeF', 'm', 0.1)


        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AutoCorrelationLayer(
                        AutoCorrelation(False, 3, attention_dropout= 0.1,
                                        output_attention=False), d_model, n_heads),
                    d_model,
                    d_ff,
                    moving_avg=3,
                    dropout=0.1,
                    activation='gelu'
                ) for l in range(2)
            ],
            norm_layer=my_Layernorm(d_model)
        )
        # Decoder
        self.decoder = Decoder(
            [
                DecoderLayer(
                    AutoCorrelationLayer(
                        AutoCorrelation(True, 3, attention_dropout= 0.1,
                                        output_attention=False), d_model, n_heads),
                    AutoCorrelationLayer(
                        AutoCorrelation(False, 3, attention_dropout= 0.1,
                                        output_attention=False), d_model, n_heads),
                    d_model,
                    1,
                    d_ff,
                    moving_avg= 3,
                    dropout=0.1,
                    activation='gelu',
                )
                for l in range(1)
            ],
            norm_layer=my_Layernorm(d_model),
            projection=nn.Linear(d_model, 1, bias=True)
        )
        self.fc = nn.Linear(1,1)
        self.act = nn.Tanh()

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec,
                enc_self_mask=None, dec_self_mask=None, dec_enc_mask=None):
        # decomp init
        mean = torch.mean(x_enc, dim=1).unsqueeze(1).repeat(1, 2, 1) #(batch_size, 1, 1)
        zeros = torch.zeros([x_dec.shape[0], 2, x_dec.shape[2]], device=x_enc.device) #(batch_size, 1, 1)
        seasonal_init, trend_init = self.decomp(x_enc[:, :3, :]) #(batch_size, 5 ,1)
        # decoder input
        trend_init = torch.cat([trend_init, mean], dim=1) #(batch_size, 5, 1)
        seasonal_init = torch.cat([seasonal_init, zeros], dim=1) #(batch_size, 5, 1)
        # enc
        enc_out = self.enc_embedding(x_enc, x_mark_enc) #(batch_size, 5, d_model)
        enc_out, attns = self.encoder(enc_out, attn_mask=enc_self_mask)
        # dec
        dec_out = self.dec_embedding(seasonal_init, x_mark_dec) #(batch_size, 5, d_model)
        seasonal_part, trend_part = self.decoder(dec_out, enc_out, x_mask=dec_self_mask, cross_mask=dec_enc_mask,
                                                 trend=trend_init)
        # final
        dec_out = trend_part + seasonal_part
        dec_out = dec_out[:, -1:, -1:]
        dec_out = self.fc(dec_out)
        dec_out = self.act(dec_out)

        return dec_out

## Female

In [ ]:
#Create the dataloader for female (standardize)
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

X_train = train_fm_5.iloc[5:,3:]
Y_train = torch.tensor(train_fm_5['K2'][5:].values).to(torch.float32).unsqueeze(1)

X_valid = valid_fm.iloc[:,3:]
Y_valid = torch.tensor(valid_fm['K2'].values).to(torch.float32).unsqueeze(1)

X_test = test_fm.iloc[:,3:]
Y_test = torch.tensor(test_fm['K2'].values).to(torch.float32).unsqueeze(1)

# unmark the following code for the hyperparameter tuning
#X_hyper = fm_params.iloc[5:,5:]
#Y_hyper = torch.tensor(fm_params['K2'][5:].values).to(torch.float32).unsqueeze(1)



from sklearn.preprocessing import StandardScaler

# Create an instance of the StandardScaler
scaler = StandardScaler()

# Fit the scaler to your training data
scaler.fit(X_train)
X_train_scaled = torch.tensor(scaler.transform(X_train)).to(torch.float32).unsqueeze(2)
X_valid_scaled = torch.tensor(scaler.transform(X_valid)).to(torch.float32).unsqueeze(2)
X_test_scaled = torch.tensor(scaler.transform(X_test)).to(torch.float32).unsqueeze(2)
#X_hyper_scaled = torch.tensor(scaler.transform(X_hyper)).to(torch.float32).unsqueeze(2)


batch_size = 10

# The code for autoformer and can be use for the data of different people
X_train_mark = torch.tensor(train_mark.iloc[5:,1:].values).to(torch.float32).unsqueeze(2)
X_valid_mark = torch.tensor(valid_mark.iloc[:,1:].values).to(torch.float32).unsqueeze(2)
X_test_mark = torch.tensor(test_mark.iloc[:,1:].values).to(torch.float32).unsqueeze(2)


#train = TensorDataset(X_train_scaled, Y_train)
train = TensorDataset(X_train_scaled, Y_train, X_train_mark)
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)

#valid = TensorDataset(X_valid_scaled, Y_valid)
valid = TensorDataset(X_valid_scaled, Y_valid, X_valid_mark)
valid_loader = DataLoader(valid, batch_size = batch_size, shuffle = False)

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV
import torch.optim as optim
from skorch import NeuralNetRegressor

model = NeuralNetRegressor(Autoformer,
    criterion = nn.MSELoss,
    optimizer = optim.Adam,
    batch_size = 10,
    max_epochs = 50,
    verbose=False
)

param_grid = {
    'optimizer__lr': [0.001, 0.01, 0.1],
    'module__activation':[nn.ReLU, nn.Tanh]
}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,cv=5)


grid_result = grid.fit(X_hyper_scaled,Y_hyper)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -681.110087 using {'module__activation': <class 'torch.nn.modules.activation.ReLU'>, 'optimizer__lr': 0.1}


In [ ]:
# The traing process(with can be used on all the NN models except Autoformer)
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

model = LSTM(10, 3, 0.3, nn.Tanh).to(device)
criterion = nn.MSELoss()
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
valid_loss_min = np.Inf
counter = 0

n_iters = 5000
num_epochs = n_iters / (len(X_train_scaled) / batch_size)
num_epochs = int(num_epochs)

for epochs in range(num_epochs):
    model.train()

    for i, (data, labels) in enumerate(train_loader):
        counter += 1
        data = data.to(device)
        labels = labels.to(device)

        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if counter % 50 == 0:
            val_losses = []
            model.eval()
            with torch.no_grad():
              for data, labels in valid_loader:
                data = data.to(device)
                labels = labels.to(device)
                outputs = model(data)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())

            model.train()
            print("Epoch: {}/{}...".format(epochs+1, num_epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

            if np.mean(val_losses) < valid_loss_min:
                torch.save(model.state_dict(), './state_dict_biLSTM.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

In [ ]:
# The training process for Autoformer
import torch.optim as optim
model = Autoformer(256,8,512).to(device)
criterion = nn.MSELoss()
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
valid_loss_min = np.Inf
counter = 0

n_iters = 1000
num_epochs = n_iters / (len(X_train_scaled) / batch_size)
num_epochs = int(num_epochs)

for epochs in range(num_epochs):
    model.train()
    counter += 1
    for i, (data, labels, mark) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.float().to(device).unsqueeze(2)

        mark = mark.float().to(device)
        # decoder input
        dec_inp = torch.zeros_like(labels[:, -1:, :]).float()
        dec_inp = torch.cat([labels[:, :4, :], dec_inp], dim=1).float().to(device)

        outputs = model(data, mark, dec_inp, mark)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    val_losses = []
    model.eval()
    with torch.no_grad():
        for data, labels, mark in valid_loader:
                data = data.to(device)
                labels = labels.float().unsqueeze(2).to(device)

                mark = mark.float().to(device)
                # decoder input
                dec_inp = torch.zeros_like(labels[:, -1:, :]).float()
                dec_inp = torch.cat([labels[:, :4, :], dec_inp], dim=1).float().to(device)

                outputs = model(data, mark, dec_inp, mark)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())

    model.train()
    print("Epoch: {}/{}...".format(epochs+1, num_epochs),
          "Loss: {:.6f}...".format(loss.item()),
          "Val Loss: {:.6f}".format(np.mean(val_losses)))

    if np.mean(val_losses) < valid_loss_min:
                torch.save(model.state_dict(), './state_dict_autoformer.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
                counter = 0
    elif np.mean(val_losses) >= valid_loss_min and counter == 20:
      break

Epoch: 1/172... Loss: 0.131253... Val Loss: 0.120781
Validation loss decreased (inf --> 0.120781).  Saving model ...
Epoch: 2/172... Loss: 0.159781... Val Loss: 0.145391
Epoch: 3/172... Loss: 0.132209... Val Loss: 0.127847
Epoch: 4/172... Loss: 0.073526... Val Loss: 0.085546
Validation loss decreased (0.120781 --> 0.085546).  Saving model ...
Epoch: 5/172... Loss: 0.071521... Val Loss: 0.085698
Epoch: 6/172... Loss: 0.097469... Val Loss: 0.100675
Epoch: 7/172... Loss: 0.117522... Val Loss: 0.101494
Epoch: 8/172... Loss: 0.133785... Val Loss: 0.105014
Epoch: 9/172... Loss: 0.141538... Val Loss: 0.110679
Epoch: 10/172... Loss: 0.138911... Val Loss: 0.101212
Epoch: 11/172... Loss: 0.140086... Val Loss: 0.092801
Epoch: 12/172... Loss: 0.147709... Val Loss: 0.099080
Epoch: 13/172... Loss: 0.171034... Val Loss: 0.124086
Epoch: 14/172... Loss: 0.168476... Val Loss: 0.118868
Epoch: 15/172... Loss: 0.150327... Val Loss: 0.102330
Epoch: 16/172... Loss: 0.156557... Val Loss: 0.108953
Epoch: 17/17

In [ ]:
# generate the prediciton output(exclude Autofomer)
model.load_state_dict(torch.load('./state_dict_biLSTM.pt'))

model.eval()
X_test = X_test_scaled.to(device)
Y_test = Y_test.to(device)
output = model(X_test)
output = output.detach().cpu().numpy()

In [ ]:
# this code use to generate the prediction for the training interval(exclude Autoformer)
model.load_state_dict(torch.load('./state_dict_biLSTM.pt'))

model.eval()
X_train = X_train_scaled.to(device)
output = model(X_train)
output = output.detach().cpu().numpy()
Y_train_m = Y_train_m.detach().cpu().numpy()

In [ ]:
# generate the prediction output for Autoformer
model.load_state_dict(torch.load('./state_dict_autoformer.pt'))

model.eval()
X_test = X_test_scaled.to(device)
#Y_test = Y_test.to(device)
Y_test = Y_test.to(device).unsqueeze(2)
mark = X_test_mark.float().to(device)
#decoder input
dec_inp = torch.zeros_like(Y_test[:, -1:, :]).float()
dec_inp = torch.cat([Y_test[:, :4, :], dec_inp], dim=1).float().to(device)

output = model(X_test, mark, dec_inp, mark)
#output = model(X_test)
output = output.squeeze(2).detach().cpu().numpy()

In [ ]:
# this code use to generate the prediction for the training interval for Autoformer
model.load_state_dict(torch.load('./state_dict_autoformer.pt'))
model.eval()
X_train = X_train_scaled.to(device)
Y_train = Y_train.to(device).unsqueeze(2)
mark = X_train_mark.float().to(device)
dec_inp = torch.zeros_like(Y_train[:, -1:, :]).float()
dec_inp = torch.cat([Y_train[:, :4, :], dec_inp], dim=1).float().to(device)

output = model(X_train, mark, dec_inp, mark)
output = output.squeeze(2).detach().cpu().numpy()
Y_train = Y_train.squeeze(2).detach().cpu().numpy()

#### Create prediciton interval (all the NN model follow the same step)

In [ ]:
# the training error
error_var = (1/57) * np.sum((Y_train-output)**2)

In [ ]:
# store the prediction output generated from different sample
autoformer_fm = pd.DataFrame(output, columns = ['s'])

In [ ]:
autoformer_fm['s5'] = output

In [ ]:
autoformer_fm['average'] = autoformer_fm.mean(axis=1)

In [ ]:
# bagged estimation for variance
var = 0
for i in range(6):
  var += (autoformer_fm.iloc[:,i].values - autoformer_fm['average'].values[:,]) ** 2

autoformer_fm['k_var'] =  1/5 * var

In [ ]:
# generate the output
autoformer_fm['lower_bound'] = autoformer_fm['average'].values + 1.96 * np.sqrt(autoformer_fm['k_var'].values + error_var)
autoformer_fm['upper_bound'] = autoformer_fm['average'].values - 1.96 * np.sqrt(autoformer_fm['k_var'].values + error_var)

In [ ]:
autoformer_fm.to_csv('autoformer_fm_2.csv', index=False)

## Male

In [ ]:
#Create the dataloader for male
X_train_m = train_m_5.iloc[5:,3:]
Y_train_m = torch.tensor(train_m_5['K2'][5:].values).to(torch.float32).unsqueeze(1)

X_valid_m = valid_m.iloc[:,3:]
Y_valid_m = torch.tensor(valid_m['K2'].values).to(torch.float32).unsqueeze(1)

X_test_m = test_m.iloc[:,3:]
Y_test_m = torch.tensor(test_m['K2'].values).to(torch.float32).unsqueeze(1)

#X_hyper_m = m_params.iloc[5:,4:]
#Y_hyper_m = torch.tensor(m_params['K2'][5:].values).to(torch.float32).unsqueeze(1)

In [ ]:
# Fit the scaler to your training data
scaler.fit(X_train_m)
X_train_scaled_m = torch.tensor(scaler.transform(X_train_m)).to(torch.float32).unsqueeze(2)
X_valid_scaled_m = torch.tensor(scaler.transform(X_valid_m)).to(torch.float32).unsqueeze(2)
X_test_scaled_m = torch.tensor(scaler.transform(X_test_m)).to(torch.float32).unsqueeze(2)
#X_hyper_scaled_m = torch.tensor(scaler.transform(X_hyper_m)).to(torch.float32).unsqueeze(2)

batch_size = 10


#train = TensorDataset(X_train_scaled_m,Y_train_m)
train = TensorDataset(X_train_scaled_m, Y_train_m, X_train_mark) # the X_train_mark is same as the mark generate in Female part
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)

#valid = TensorDataset(X_valid_scaled_m,Y_valid_m)
valid = TensorDataset(X_valid_scaled_m, Y_valid_m, X_valid_mark) # the X_valid_mark is same as the mark generate in Female part
valid_loader = DataLoader(valid, batch_size = batch_size, shuffle = False)

In [ ]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV
import torch.optim as optim
from skorch import NeuralNetRegressor

model = NeuralNetRegressor(LTSF_N,
    criterion = nn.MSELoss,
    optimizer = optim.Adam,
    batch_size = 10,
    max_epochs = 100,
    verbose=False
)

param_grid = {
    'optimizer__lr': [0.001, 0.01, 0.1],
    'module__activation':[nn.ReLU, nn.Tanh]
}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,cv=5)


grid_result = grid.fit(X_hyper_scaled_m,Y_hyper_m)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -5.487098 using {'module__activation': <class 'torch.nn.modules.activation.ReLU'>, 'optimizer__lr': 0.1}


In [ ]:
# The traing process(with can be used on all the NN models except Autoformer)
model = biLSTM(10, 3, 0.3, nn.Tanh).to(device)
criterion = nn.MSELoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
valid_loss_min = np.Inf
counter = 0

n_iters = 5000
num_epochs = n_iters / (len(X_train_m) / batch_size)
num_epochs = int(num_epochs)

for epochs in range(num_epochs):
    model.train()
    for i, (data, labels) in enumerate(train_loader):
        counter += 1
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if counter % 50 == 0:
            val_losses = []
            model.eval()
            with torch.no_grad():
               for data, labels in valid_loader:
                 data = data.to(device)
                 labels = labels.to(device)
                 outputs = model(data)
                 loss = criterion(outputs, labels)
                 val_losses.append(loss.item())

            model.train()
            print("Epoch: {}/{}...".format(epochs+1, num_epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

            if np.mean(val_losses) < valid_loss_min:
                torch.save(model.state_dict(), './state_dict_biLSTM-m.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

In [ ]:
# The training process for Autoformer
import torch.optim as optim
model = Autoformer(256,4,128).to(device)
criterion = nn.MSELoss()
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1)
valid_loss_min = np.Inf
counter = 0

n_iters = 1000
num_epochs = n_iters / (len(X_train_scaled_m) / batch_size)
num_epochs = int(num_epochs)

for epochs in range(num_epochs):
    model.train()
    counter += 1
    for i, (data, labels, mark) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.float().to(device).unsqueeze(2)

        mark = mark.float().to(device)
        # decoder input
        dec_inp = torch.zeros_like(labels[:, -1:, :]).float()
        dec_inp = torch.cat([labels[:, :4, :], dec_inp], dim=1).float().to(device)

        outputs = model(data, mark, dec_inp, mark)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    val_losses = []
    model.eval()
    with torch.no_grad():
        for data, labels, mark in valid_loader:
                data = data.to(device)
                labels = labels.float().unsqueeze(2).to(device)

                mark = mark.float().to(device)
                # decoder input
                dec_inp = torch.zeros_like(labels[:, -1:, :]).float()
                dec_inp = torch.cat([labels[:, :4, :], dec_inp], dim=1).float().to(device)

                outputs = model(data, mark, dec_inp, mark)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())

    model.train()
    print("Epoch: {}/{}...".format(epochs+1, num_epochs),
          "Loss: {:.6f}...".format(loss.item()),
          "Val Loss: {:.6f}".format(np.mean(val_losses)))

    if np.mean(val_losses) < valid_loss_min:
                torch.save(model.state_dict(), './state_dict_autoformer_m.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
                counter = 0
    elif np.mean(val_losses) >= valid_loss_min and counter == 20:
      break
    #scheduler.step()

Epoch: 1/172... Loss: 0.088105... Val Loss: 0.087457
Validation loss decreased (inf --> 0.087457).  Saving model ...
Epoch: 2/172... Loss: 0.085148... Val Loss: 0.075473
Validation loss decreased (0.087457 --> 0.075473).  Saving model ...
Epoch: 3/172... Loss: 0.090787... Val Loss: 0.076153
Epoch: 4/172... Loss: 0.106779... Val Loss: 0.091388
Epoch: 5/172... Loss: 0.131557... Val Loss: 0.114885
Epoch: 6/172... Loss: 0.166249... Val Loss: 0.144035
Epoch: 7/172... Loss: 0.201849... Val Loss: 0.170419
Epoch: 8/172... Loss: 0.226451... Val Loss: 0.189249
Epoch: 9/172... Loss: 0.240868... Val Loss: 0.201177
Epoch: 10/172... Loss: 0.237544... Val Loss: 0.196842
Epoch: 11/172... Loss: 0.222952... Val Loss: 0.184745
Epoch: 12/172... Loss: 0.209092... Val Loss: 0.173906
Epoch: 13/172... Loss: 0.195770... Val Loss: 0.163282
Epoch: 14/172... Loss: 0.185494... Val Loss: 0.156075
Epoch: 15/172... Loss: 0.188609... Val Loss: 0.158905
Epoch: 16/172... Loss: 0.182560... Val Loss: 0.146211
Epoch: 17/17

In [ ]:
# generate the prediciton output(exclude Autofomer)
model.load_state_dict(torch.load('./state_dict_biLSTM-m.pt'))

model.eval()
X_test = X_test_scaled_m.to(device)
Y_test = Y_test_m.to(device)
output = model(X_test)
output = output.detach().cpu().numpy()

In [ ]:
# generate the prediciton output for Autoformer
model.load_state_dict(torch.load('./state_dict_autoformer_m.pt'))

model.eval()
X_test = X_test_scaled_m.to(device)
#Y_test = Y_test.to(device)
Y_test = Y_test_m.to(device).unsqueeze(2)
mark = X_test_mark.float().to(device)
#decoder input
dec_inp = torch.zeros_like(Y_test[:, -1:, :]).float()
dec_inp = torch.cat([Y_test[:, :4, :], dec_inp], dim=1).float().to(device)

output = model(X_test, mark, dec_inp, mark)
#output = model(X_test)
output = output.squeeze(2).detach().cpu().numpy()

In [ ]:
# this code use to generate the prediction for the training interval(exclude Autoformer)
model.load_state_dict(torch.load('./state_dict_biLSTM-m.pt'))

model.eval()
X_train = X_train_scaled_m.to(device)
output = model(X_train)
output = output.detach().cpu().numpy()
Y_train_m = Y_train_m.detach().cpu().numpy()

In [ ]:
# this code use to generate the prediction for the training interval for Autoformer
model.load_state_dict(torch.load('./state_dict_autoformer_m.pt'))
model.eval()
X_train = X_train_scaled_m.to(device)
Y_train = Y_train_m.to(device).unsqueeze(2)
mark = X_train_mark.float().to(device)
dec_inp = torch.zeros_like(Y_train[:, -1:, :]).float()
dec_inp = torch.cat([Y_train[:, :4, :], dec_inp], dim=1).float().to(device)

output = model(X_train, mark, dec_inp, mark)
output = output.squeeze(2).detach().cpu().numpy()
Y_train = Y_train.squeeze(2).detach().cpu().numpy()

In [ ]:
# calculate the training error
error_var = (1/57) * np.sum((Y_train-output)**2)

In [ ]:
autoformer_m = pd.DataFrame(output, columns = ['s'])

In [ ]:
autoformer_m['s5'] = output

In [ ]:
autoformer_m['average'] = autoformer_m.mean(axis=1)

In [ ]:
var = 0
for i in range(5):
  var += (autoformer_m.iloc[:,i].values - autoformer_m['average'].values[:,]) ** 2

autoformer_m['k_var'] =  1/4 * var

In [ ]:
# generate the prediction interval
autoformer_m['lower_bound'] = autoformer_m['average'].values + 1.96 * np.sqrt(autoformer_m['k_var'].values + error_var)
autoformer_m['upper_bound'] = autoformer_m['average'].values - 1.96 * np.sqrt(autoformer_m['k_var'].values + error_var)

In [ ]:
autoformer_m.to_csv('autoformer_m_2.csv', index=False)

## Total

In [ ]:
#Create the dataloader for total (standardized)
X_train_t = train_t.iloc[5:,3:]
Y_train_t = torch.tensor(train_t['K2'][5:].values).to(torch.float32).unsqueeze(1)

X_valid_t = valid_t.iloc[:,3:]
Y_valid_t = torch.tensor(valid_t['K2'].values).to(torch.float32).unsqueeze(1)

X_test_t = test_t.iloc[:,3:]
Y_test_t = torch.tensor(test_t['K2'].values).to(torch.float32).unsqueeze(1)

#X_hyper_t = t_params.iloc[5:,5:]
#Y_hyper_t = torch.tensor(t_params['K2'][5:].values).to(torch.float32).unsqueeze(1)

In [ ]:
# Fit the scaler to your training data
scaler.fit(X_train_t)
X_train_scaled_t = torch.tensor(scaler.transform(X_train_t)).to(torch.float32).unsqueeze(2)
X_valid_scaled_t = torch.tensor(scaler.transform(X_valid_t)).to(torch.float32).unsqueeze(2)
X_test_scaled_t = torch.tensor(scaler.transform(X_test_t)).to(torch.float32).unsqueeze(2)
#X_hyper_scaled_t = torch.tensor(scaler.transform(X_hyper_t)).to(torch.float32).unsqueeze(2)

batch_size = 10


#train = TensorDataset(X_train_scaled_t,Y_train_t)
train = TensorDataset(X_train_scaled_t, Y_train_t, X_train_mark) # the X_train_mark is same as the mark generate in Female part
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)

#valid = TensorDataset(X_valid_scaled_t,Y_valid_t)
valid = TensorDataset(X_valid_scaled_t, Y_valid_t, X_valid_mark) # the X_valid_mark is same as the mark generate in Female part
valid_loader = DataLoader(valid, batch_size = 5, shuffle = False)

In [ ]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV
import torch.optim as optim
from skorch import NeuralNetRegressor

model = NeuralNetRegressor(LTSF_N,
    criterion = nn.MSELoss,
    optimizer = optim.Adam,
    batch_size = 10,
    max_epochs = 100,
    verbose=False
)

param_grid = {
    'optimizer__lr': [0.001, 0.01, 0.1],
    'module__activation':[nn.ReLU, nn.Tanh]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,cv=5)


grid_result = grid.fit(X_hyper_scaled_t,Y_hyper_t)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -1615.222297 using {'module__activation': <class 'torch.nn.modules.activation.ReLU'>, 'optimizer__lr': 0.1}


In [ ]:
 # the traing process (exclude Autoformer)
import torch.optim as optim

model = biLSTM(30, 3, 0.1, nn.Tanh).to(device)
criterion = nn.MSELoss()
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
valid_loss_min = np.Inf
counter = 0

n_iters = 5000
num_epochs = n_iters / (len(X_train_t) / batch_size)
num_epochs = int(num_epochs)

for epochs in range(num_epochs):
    model.train()
    for i, (data, labels) in enumerate(train_loader):
        counter += 1
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if counter % 50 == 0:
            val_losses = []
            model.eval()
            with torch.no_grad():
               for data, labels in valid_loader:
                 data = data.to(device)
                 labels = labels.to(device)
                 outputs = model(data)
                 loss = criterion(outputs, labels)
                 val_losses.append(loss.item())

            model.train()
            print("Epoch: {}/{}...".format(epochs+1, num_epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

            if np.mean(val_losses) < valid_loss_min:
                torch.save(model.state_dict(), './state_dict_biLSTM-t.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

In [ ]:
# the training process for Autoformer
import torch.optim as optim
model = Autoformer(256,4,128).to(device)
criterion = nn.MSELoss()
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1)
valid_loss_min = np.Inf
counter = 0

n_iters = 1000
num_epochs = n_iters / (len(X_train_scaled_t) / batch_size)
num_epochs = int(num_epochs)

for epochs in range(num_epochs):
    model.train()
    counter += 1
    for i, (data, labels, mark) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.float().to(device).unsqueeze(2)

        mark = mark.float().to(device)
        # decoder input
        dec_inp = torch.zeros_like(labels[:, -1:, :]).float()
        dec_inp = torch.cat([labels[:, :4, :], dec_inp], dim=1).float().to(device)

        outputs = model(data, mark, dec_inp, mark)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    val_losses = []
    model.eval()
    with torch.no_grad():
        for data, labels, mark in valid_loader:
                data = data.to(device)
                labels = labels.float().unsqueeze(2).to(device)

                mark = mark.float().to(device)
                # decoder input
                dec_inp = torch.zeros_like(labels[:, -1:, :]).float()
                dec_inp = torch.cat([labels[:, :4, :], dec_inp], dim=1).float().to(device)

                outputs = model(data, mark, dec_inp, mark)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())

    model.train()
    print("Epoch: {}/{}...".format(epochs+1, num_epochs),
          "Loss: {:.6f}...".format(loss.item()),
          "Val Loss: {:.6f}".format(np.mean(val_losses)))

    if np.mean(val_losses) < valid_loss_min:
                torch.save(model.state_dict(), './state_dict_autoformer_t.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
                counter = 0
    elif np.mean(val_losses) >= valid_loss_min and counter == 20:
      break
    #scheduler.step()

Epoch: 1/172... Loss: 0.693696... Val Loss: 0.710832
Validation loss decreased (inf --> 0.710832).  Saving model ...
Epoch: 2/172... Loss: 0.623723... Val Loss: 0.693104
Validation loss decreased (0.710832 --> 0.693104).  Saving model ...
Epoch: 3/172... Loss: 0.585070... Val Loss: 0.676705
Validation loss decreased (0.693104 --> 0.676705).  Saving model ...
Epoch: 4/172... Loss: 0.587045... Val Loss: 0.673341
Validation loss decreased (0.676705 --> 0.673341).  Saving model ...
Epoch: 5/172... Loss: 0.596236... Val Loss: 0.673470
Epoch: 6/172... Loss: 0.611472... Val Loss: 0.671064
Validation loss decreased (0.673341 --> 0.671064).  Saving model ...
Epoch: 7/172... Loss: 0.623266... Val Loss: 0.665524
Validation loss decreased (0.671064 --> 0.665524).  Saving model ...
Epoch: 8/172... Loss: 0.670074... Val Loss: 0.648882
Validation loss decreased (0.665524 --> 0.648882).  Saving model ...
Epoch: 9/172... Loss: 0.714087... Val Loss: 0.641856
Validation loss decreased (0.648882 --> 0.641

In [ ]:
# generate the prediciton output(exclude Autofomer)
model.load_state_dict(torch.load('./state_dict_biLSTM-t.pt'))

model.eval()
X_test = X_test_scaled_t.to(device)
Y_test = Y_test_t.to(device)
output = model(X_test)
output = output.detach().cpu().numpy()

In [ ]:
# generate the prediciton output for Autoformer
model.load_state_dict(torch.load('./state_dict_autoformer_t.pt'))

model.eval()
X_test = X_test_scaled_t.to(device)
#Y_test = Y_test.to(device)
Y_test = Y_test_t.to(device).unsqueeze(2)
mark = X_test_mark.float().to(device)
#decoder input
dec_inp = torch.zeros_like(Y_test[:, -1:, :]).float()
dec_inp = torch.cat([Y_test[:, :4, :], dec_inp], dim=1).float().to(device)

output = model(X_test, mark, dec_inp, mark)
#output = model(X_test)
output = output.squeeze(2).detach().cpu().numpy()

In [ ]:
# this code use to generate the prediction for the training interval(exclude Autoformer)
model.load_state_dict(torch.load('./state_dict_biLSTM-t.pt'))

model.eval()
X_train = X_train_scaled_t.to(device)
output = model(X_train)
output = output.detach().cpu().numpy()
Y_train_t = Y_train_t.detach().cpu().numpy()

In [ ]:
# this code use to generate the prediction for the training interval for Autoformer
model.load_state_dict(torch.load('./state_dict_autoformer_t.pt'))
model.eval()
X_train = X_train_scaled_t.to(device)
Y_train = Y_train_t.to(device).unsqueeze(2)
mark = X_train_mark.float().to(device)
dec_inp = torch.zeros_like(Y_train[:, -1:, :]).float()
dec_inp = torch.cat([Y_train[:, :4, :], dec_inp], dim=1).float().to(device)

output = model(X_train, mark, dec_inp, mark)
output = output.squeeze(2).detach().cpu().numpy()
Y_train = Y_train.squeeze(2).detach().cpu().numpy()

In [ ]:
# generate the traing error
error_var = (1/57) * np.sum((Y_train-output)**2)

In [ ]:
autoformer_t = pd.DataFrame(output, columns = ['s'])

In [ ]:
autoformer_t['s5'] = output

In [ ]:
autoformer_t['average'] = autoformer_t.mean(axis=1)

In [ ]:
var = 0
for i in range(6):
  var += (autoformer_t.iloc[:,i].values - autoformer_t['average'].values[:,]) ** 2

autoformer_t['k_var'] =  1/5 * var

In [ ]:
# generate the prediction interval
autoformer_t['lower_bound'] = autoformer_t['average'].values + 1.96 * np.sqrt(autoformer_t['k_var'].values + error_var)
autoformer_t['upper_bound'] = autoformer_t['average'].values - 1.96 * np.sqrt(autoformer_t['k_var'].values + error_var)

In [ ]:
autoformer_t.to_csv('autoformer_t_2.csv', index=False)